Mark Solutions and Benchmark the Results

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
def read_sirius_output(tsv_path):
    selected_columns = ["smiles", "CSI:FingerIDScore", "name"]
    sirius_df = pd.read_csv(f"{tsv_path}", sep='\t', usecols=selected_columns)
    
    return sirius_df

In [ ]:
from fiora.MOL.Metabolite import Metabolite

def mark_casmi_solutions(df, solution_df):
    df["source"] = "Candidate"
        
    # df['InChIKey_split'] = None
    for i, row in solution_df.iterrows():
        challenge_name = row["ChallengeName"]
        metabolite = Metabolite(row["SMILES"])  # Metabolite i df e kaydet
        # inchikey_2d = inchikey.split('-', 1)[0]
        # df['InChIKey_split'] = df['InChIKey'].str.split('-', expand=True)[0]
        condition = (df["Challenge"] == challenge_name) & (df["Metabolite"] == metabolite)
        df.loc[condition, "source"] = "Solution"
        print(df.loc[condition, "CompoundName"])
        
def mark_sirius_solutions(df, solution_df, challenge_no):
    df["source"] = "Candidate"
    try:
        challenge_solution_row = solution_df.loc[solution_df["ChallengeName"] == f"Challenge-{challenge_no}"].iloc[0]
    except Exception as e:
        print(e)
    solution_inchi_2d = challenge_solution_row["INCHIKEY"].split('-', 1)[0]
    condition = df["InChIkey2D"] == solution_inchi_2d
    df.loc[condition, "source"] = "Solution"

In [ ]:
fiora_output_df = pd.read_csv("/home/lbarbut/scored_outputs/test_neg_cand.csv")
fiora_output_df.sort_values(by="CosineScore", ascending=False, na_position="last", inplace=True)

In [ ]:
# fiora_scores = pd.read_csv("scored_output.csv")     # list according to score
# sirios_scores = read_sirius_output("structure_candidates.tsv")

In [ ]:

fiora_output_df.dropna(subset=['Metabolite'], inplace=True)
fiora_output_df

In [ ]:
# 1-2-3-4-5-10 Recall
def calculate_recall_at_k(prediction_df, k):    # für alle Challenges zusammen, +100 Solutions
    top_k_predictions = prediction_df.head(k)
    return sum(top_k_predictions['source'] == 'Solution') / sum(prediction_df['source'] == 'Solution')  # len statt solution

solution_df = pd.read_csv("/home/lbarbut/.vscode-server/data/CASMI_2016/solutions_casmi2016_cat2and3.csv")
mark_casmi_solutions(fiora_output_df, solution_df)

# recall_at_5 = calculate_recall_at_k(fiora_output_df, 5)
# recall_at_100 = calculate_recall_at_k(fiora_output_df, 100)

# print(f'Recall@5: {recall_at_5}')
# print(f'Recall@100: {recall_at_100}')



In [ ]:
fiora_output_df["Metabolite"]

In [ ]:
fiora_output_df["source"].value_counts()

In [ ]:
k=100000
top_k_predictions =fiora_output_df.groupby("Challenge").head(k)

#sum(top_k_predictions['source'] == "Solution") / len(fiora_output_df["Challenge"].unique())

In [ ]:
fiora_output_df[fiora_output_df['source'] == "Solution"][["Challenge", "SMILES"]]

In [ ]:
from fiora.MOL.Metabolite import Metabolite
import fiora.visualization.spectrum_visualizer as sv
import ast



spec = ast.literal_eval(fiora_output_df.loc[719].Spectrum)
sv.plot_spectrum({'peaks': spec})
#Metabolite("c1cc(nc(c1Cl)C(=O)O)Cl").draw()

In [ ]:
tp = 0
for c in fiora_output_df["Challenge"].unique():
    if any(fiora_output_df[fiora_output_df["Challenge"] == c]['source'].head(k) == "Solution"):
        print(c)
        tp += 1
tp / len(fiora_output_df["Challenge"].dropna().unique())

In [ ]:
any(fiora_output_df[fiora_output_df["Challenge"] == "Challenge-008"]['source'].head(k) == "Solution")

In [ ]:
fiora_output_df.groupby("Challenge")['source']